In [ ]:
import sys
import numpy as np
import time
import matplotlib.pyplot as plt
sys.path.append("C:/Users/Administrator/Downloads")  # PATH to hardware_ops.py

In [ ]:
from hardware_ops import HardwareOps

config_filename =  r'C:\Users\Administrator\Downloads\Research\hardware_config.json' #use this if you ned to store serial port numbers
ops = HardwareOps(config_filename)#initialized inside the __init__ functions

## Piezo Jacobian

In [ ]:
piezo_list = np.array(ops.piezo_serials_list)
y_piezo = piezo_list[[1]]
x_piezo = piezo_list[[0]]
print(ops.piezos.get_all_voltages())

In [ ]:
ops.piezos.set_voltage(y_piezo[0], 0)
ops.piezos.set_voltage(x_piezo[0], 0)

In [ ]:
def piezo_Jacobian(piezo_list, step_size, settle_time=0.5):
    y_piezo = piezo_list[[1]]
    x_piezo = piezo_list[[0]]
    axes = [
        ("Y", y_piezo[0]),
        ("X", x_piezo[0])
    ]
    Mirror = []  # positions after positive moves

    for axis_name, axis_channel in axes:

        idx = np.where(piezo_list == axis_channel)[0][0]
        ## Movement
        # Mirror Pos
        pos_before = ops.quads.get_xy_position(sig_strength=0.02)
        Mirror.append(pos_before)

        ops.piezos.set_voltage(axis_channel, pos_before[idx] + step_size) # Move + step
        time.sleep(settle_time)

        pos_after = ops.quads.get_xy_position(sig_strength=0.02)
        Mirror.append(pos_after)

        # Move back (Neg)
        ops.piezos.set_voltage(axis_channel, pos_before[idx])
        time.sleep(settle_time)

        neg_after = ops.quads.get_xy_position(sig_strength=0.02)
        Mirror.append(neg_after)

    m = []

    pairs = [(1,0), (2,1), (3,2), (4,3)]

    for a_idx, b_idx in pairs:
        a = Mirror[a_idx] # after
        b = Mirror[b_idx] # before
        m.append([x - y for x, y in zip(a,b)])

    M = np.array(m)*2/step_size # Doubling 'cause of mirror law & dividing by initial step jump: pM is the linear movement of laser per singular input

    return m

In [ ]:
import numpy as np
import time

def piezo_Jacobian_stepwise(piezo_list, step_size, settle_time=0.5):
    """
    Moves piezo actuators in steps of 1 towards step_size, recording XY position at each step.
    
    Parameters:
        piezo_list (list/array): [X_channel, Y_channel]
        step_size (int): total voltage change to apply
        settle_time (float): wait time after each step
    
    Returns:
        Mirror_steps (dict): recorded XY positions for each axis at every step
            Example: {"X": [positions], "Y": [positions]}
    """
    
    y_piezo = piezo_list[1]
    x_piezo = piezo_list[0]
    
    axes = [("X", x_piezo), ("Y", y_piezo)]
    
    Mirror_steps = {"X": [], "Y": []}
    
    for axis_name, axis_channel in axes:

        idx = np.where(piezo_list == axis_channel)[0][0]

        volt_start = ops.piezos.get_all_voltages()[idx]

        pos_start = ops.quads.get_xy_position(sig_strength=0.02) # 0
        Mirror_steps[axis_name].append(pos_start)
        
        # Determine step direction
        step_dir = 1 if step_size > 0 else -1
        
        # --- Ramp up: move incrementally to step_size --- 1-5
        for step in range(1, abs(step_size) + 1):
            new_volt = volt_start + step * step_dir
            ops.piezos.set_voltage(axis_channel, new_volt)
            time.sleep(settle_time)
            pos = ops.quads.get_xy_position(sig_strength=0.02)
            Mirror_steps[axis_name].append(pos)

        # --- Ramp down: move incrementally back to start --- 5-0
        for step in range(abs(step_size) - 1, -1, -1):  # decrement steps
            new_volt = volt_start + step * step_dir
            ops.piezos.set_voltage(axis_channel, new_volt)
            time.sleep(settle_time)
            pos = ops.quads.get_xy_position(sig_strength=0.02)
            Mirror_steps[axis_name].append(pos)
        
        # Move back to starting position
        ops.piezos.set_voltage(axis_channel, volt_start)
        time.sleep(settle_time)

    m = []
    for axis_name in ["X", "Y"]:
        for i in range(1, len(Mirror_steps[axis_name])):
            M = [x - y for x, y in zip(Mirror_steps[axis_name][i], Mirror_steps[axis_name][i-1])]
            m.append(M)

    return m

    # m = []
    # for i in range(1, len(Mirror_steps[axis_name])):
    #     M = [x - y for x, y in zip(Mirror_steps[axis_name][i], Mirror_steps[axis_name][i-1])]
    #     m.append(M)

# Mirror1 = [m₁, m₂, ..., m₁₅₀]
# Mirror2 = [...]
# Mirror3 = [...]


In [ ]:
Mirror_data = {
    # "Mirror1": [[] for _ in range(4)],
    # "Mirror2": [[] for _ in range(4)],
    # "Mirror3": [[] for _ in range(4)]
    "Mirror0": [[]for _ in range(4)]
}
step_size = 5

In [ ]:
for run in range(10):
    # m = piezo_Jacobian(piezo_list, step_size)
    m = piezo_Jacobian_stepwise(piezo_list, step_size)

    for i in range(4):
        Mirror_data["Mirror0"][i].append(m[i]) #Change Mirror_ For each run

Plotting Code is Questionable: Work in Progress

In [ ]:
# Run after all runs are stored
Mirror1 = np.array(Mirror_data['Mirror1'])
Mirror2 = np.array(Mirror_data['Mirror2'])
Mirror3 = np.array(Mirror_data['Mirror3'])

import matplotlib.pyplot as plt
import numpy as np

datasets = [Mirror1, Mirror2, Mirror3]
labels = ['Mirror1', 'Mirror2', 'Mirror3']
colors = ['r', 'g', 'b']

for group in range(4):
    plt.figure(figsize=(10,6))
    for i, data in enumerate(datasets):
        for feature in range(4):
            plt.plot(data[group,:,feature], color=colors[i], linestyle=['-', '--', ':', '-.'][feature], label=f'{labels[i]} F{feature+1}' if feature==0 else "")
    plt.title(f'Group {group+1} Features over Time')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.show()


In [ ]:
for i in range(Mirror2.shape[0]):
    for j in range(Mirror2.shape[1]):
        plt.plot(Mirror2[i, j])
plt.show()

for i in range(Mirror3.shape[0]):
    for j in range(Mirror3.shape[1]):
        plt.plot(Mirror3[i, j])  
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_fine_response(Mirror_data, num_runs=50):
    """
    Plots mirror responses over multiple runs.
    
    Parameters:
        Mirror_data (dict): Dictionary with keys "Mirror1", "Mirror2", "Mirror3".
                            Each value is a list of 4 lists, corresponding to axes.
                            Example: Mirror_data["Mirror1"][0] = list of X responses.
        num_runs (int): Number of runs to plot.
    """
    
    # Convert to numpy arrays for easier handling
    Mirror1 = np.array(Mirror_data["Mirror1"])  # shape (4, num_runs)
    Mirror2 = np.array(Mirror_data["Mirror2"])
    Mirror3 = np.array(Mirror_data["Mirror3"])
    
    mirrors = [Mirror1, Mirror2, Mirror3]
    mirror_labels = ["Mirror1", "Mirror2", "Mirror3"]
    axes_labels = ["X", "Y"]
    
    runs = np.arange(1, num_runs + 1)
    
    fig, axs = plt.subplots(2, 3, figsize=(15, 8))
    fig.suptitle("Mirror Responses Over Runs", fontsize=16)
    
    for col, mirror in enumerate(mirrors):
        for row, axis in enumerate([0, 1]):  # X = 0, Y = 1
            axs[row, col].plot(runs, mirror[axis], marker='o', linestyle='-', alpha=0.7)
            axs[row, col].set_title(f"{mirror_labels[col]} {axes_labels[row]}")
            axs[row, col].set_xlabel("Run")
            axs[row, col].set_ylabel("Response")
            axs[row, col].grid(True)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


In [ ]:
mirrors = {
    "Mirror1": np.array(Mirror1),
    "Mirror2": np.array(Mirror2),
    "Mirror3": np.array(Mirror3)
}

movement_labels = ["+Y", "-Y", "+X", "-X"]
q_labels = ["Q1x", "Q1y", "Q2x", "Q2y"]
colors = ["red", "blue", "green"]

# Plotting
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for qi in range(4):  # Q1x, Q1y, Q2x, Q2y
    ax = axes[qi]
    
    for (mname, mdata), color in zip(mirrors.items(), colors):
        # Extract values for this Q across movements
        y_vals = mdata[:, qi]     # 4 movements
        x_vals = np.arange(len(y_vals))
        
        ax.plot(x_vals, y_vals, marker='o', label=mname + " movements", color=color)
    
    ax.set_title(q_labels[qi], fontsize=14)
    ax.set_xticks(range(4))
    ax.set_xticklabels(movement_labels)
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()

## Coarse Jacobian

stage_ser = ops.stage_serials[0]

In [ ]:
print(ops.stages.get_all_positions())

In [ ]:
channels = ['chan1', 'chan2', 'chan3', 'chan4']
step_size = 2   # your chosen step size

def compute_jacobian(channels, step_size, settle_time=0.5):

    pQ = []  # positions after positive moves
    nQ = []  # positions after negative moves

    for i, chan in enumerate(channels):
        # --- Positive move ---
        pos_before = ops.quads.get_xy_position(sig_strength=0.02)   # Initialize Pos
        pQ.append(pos_before)  
        loc_pos = ops.stages.get_all_positions() 
        pos_target = loc_pos[i] + step_size
        ops.stages.move_absolute(stage_ser, chan, pos_target)   # Move XY
        time.sleep(settle_time)
        pos_after = ops.quads.get_xy_position(sig_strength=0.02)  # Record Change
        pQ.append(pos_after)

        # --- Negative move ---
        neg_before = ops.quads.get_xy_position(sig_strength=0.02)   # Initialize Neg
        nQ.append(neg_before)
        loc_neg = ops.stages.get_all_positions()    # Initialize Loc
        neg_target = loc_neg[i] - step_size
        ops.stages.move_absolute(stage_ser, chan, neg_target)   # Move XY
        time.sleep(settle_time)
        neg_after = ops.quads.get_xy_position(sig_strength=0.02)  # Record Change
        nQ.append(neg_after)
    # For Chan 1, Chan 2, Chan 3, Chan 4

    pM = []
    nM = []

    for i in range(len(channels)):
        a = pQ[2*i+1] # after
        b = pQ[2*i+0] # before
        pM.append([pa - pb for pa, pb in zip(a,b)])

        c = nQ[2*i+1] # after
        d = nQ[2*i+0] # before
        nM.append([na - nb for na, nb in zip(c,d)])

    pM = np.array(pM)*2/step_size # Doubling 'cause of mirror law & dividing by initial step jump: pM is the linear movement of laser per singular input
    nM = np.array(nM)*2/-step_size

    print("Positive Jacobian")
    print(pM)
    print("Negative Jacobian")
    print(nM)

    pMX = [pM[0,:], pM[2,:]]
    pMY = [pM[1,:], pM[3,:]]
    nMX = [nM[0,:], nM[2,:]]
    nMY = [nM[1,:], nM[3,:]]

    print("Positive X Jacobian:")
    for row in pMX:
        print("  ".join(f"{val:8.4f}" for val in row))
    print("Positive Y Jacobian:")
    for row in pMY:
        print("  ".join(f"{val:8.4f}" for val in row))
    print("Negative X Jacobian:")
    for row in nMX:
        print("  ".join(f"{val:8.4f}" for val in row))
    print("Negative Y Jacobian:")
    for row in nMY:
        print("  ".join(f"{val:8.4f}" for val in row))
    
    return pM, nM

In [ ]:
def distribution_grid(raw_list, names):
    """
    Plot X and Y distributions for multiple mirrors in a 2x2 grid.
    
    raw_list: list of 4 datasets (each dataset = list of rows)
    names: list of 4 names corresponding to the mirrors
    """
    fig, axes = plt.subplots(4, 2, figsize=(12, 16))  # 4 rows, 2 columns

    for i, (raw, name) in enumerate(zip(raw_list, names)):
        data = np.array([np.array(row) for row in raw], dtype=float)
        dB1x, dB1y = data[:,0], data[:,1]
        dB2x, dB2y = data[:,2], data[:,3]

        # X component
        axes[i, 0].hist(dB1x, bins=40, alpha=0.5, label="Quadcell 1")
        axes[i, 0].hist(dB2x, bins=40, alpha=0.5, label="Quadcell 2")
        axes[i, 0].set_title(f"Mirror: {name} — X Distribution")
        axes[i, 0].set_xlabel("dB_x")
        axes[i, 0].set_ylabel("Frequency")
        axes[i, 0].legend()
        axes[i, 0].grid(True)

        # Y component
        axes[i, 1].hist(dB1y, bins=40, alpha=0.5, label="Quadcell 1")
        axes[i, 1].hist(dB2y, bins=40, alpha=0.5, label="Quadcell 2")
        axes[i, 1].set_title(f"Mirror: {name} — Y Distribution")
        axes[i, 1].set_xlabel("dB_y")
        axes[i, 1].set_ylabel("Frequency")
        axes[i, 1].legend()
        axes[i, 1].grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
def distribution_abs(positive_list, negative_list, names, col_index=0):
    """
    Plot one column of 8 datasets in two stacks (positive/negative).
    
    positive_list: list of 4 datasets (each dataset = list of rows)
    negative_list: list of 4 datasets (each dataset = list of rows)
    names: list of 4 names corresponding to the mirrors
    col_index: which column to plot (0-based)
    """
    fig, axes = plt.subplots(2, 4, figsize=(20, 8))  # 2 rows, 4 columns

    # Positive stack
    for i, raw in enumerate(positive_list):
        data = np.array([np.array(row) for row in raw], dtype=float)
        # overlay all B components (dB1, dB2, etc.)
        for j in range(0, data.shape[1], 2):  # assuming B1x,B1y,B2x,B2y...
            axes[0, i].hist(data[:, j+col_index], bins=40, alpha=0.5, label=f"B{j//2 + 1}")
        axes[0, i].set_title(f"{names[i]} — Positive")
        axes[0, i].set_xlabel(f"Column {col_index}")
        axes[0, i].set_ylabel("Frequency")
        axes[0, i].legend()
        axes[0, i].grid(True)

    # Negative stack
    for i, raw in enumerate(negative_list):
        data = np.array([np.array(row) for row in raw], dtype=float)
        for j in range(0, data.shape[1], 2):
            axes[1, i].hist(data[:, j+col_index], bins=40, alpha=0.5, label=f"B{j//2 + 1}")
        axes[1, i].set_title(f"{names[i]} — Negative")
        axes[1, i].set_xlabel(f"Column {col_index}")
        axes[1, i].set_ylabel("Frequency")
        axes[1, i].legend()
        axes[1, i].grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np

def compute_jacobian_from_data(mirror_data, step_size):
    """
    mirror_data : list of 4 lists
        mirror_data[0] = +Y responses (50×4)
        mirror_data[1] = -Y responses (50×4)
        mirror_data[2] = +X responses (50×4)
        mirror_data[3] = -X responses (50×4)
    step_size : scalar piezo step used in piezo_Jacobian()
    """

    # Convert lists → arrays
    posY = np.array(mirror_data[0])  # (50 × 4)
    negY = np.array(mirror_data[1])
    posX = np.array(mirror_data[2])
    negX = np.array(mirror_data[3])

    # Mean responses
    mean_dQ_posY = np.mean(posY, axis=0)
    mean_dQ_negY = np.mean(negY, axis=0)
    mean_dQ_posX = np.mean(posX, axis=0)
    mean_dQ_negX = np.mean(negX, axis=0)

    # Jacobian estimate (4×2)
    J = np.zeros((4, 2))

    # central difference for Y-column
    J[:, 0] = (mean_dQ_posY - mean_dQ_negY) / (2 * step_size)

    # central difference for X-column
    J[:, 1] = (mean_dQ_posX - mean_dQ_negX) / (2 * step_size)

    # Uncertainty (standard deviation of estimates)
    dJ_samples_Y = (posY - negY) / (2 * step_size)
    dJ_samples_X = (posX - negX) / (2 * step_size)

    J_std = np.zeros((4,2))
    J_std[:,0] = np.std(dJ_samples_Y, axis=0)
    J_std[:,1] = np.std(dJ_samples_X, axis=0)

    return J, J_std


In [ ]:
list_pos = [dM1p, dM2p, dM3p, dM4p]
list_neg = [dM1n, dM2n, dM3n, dM3n]

distribution_abs(list_pos, list_neg, names)

Tested Data

In [ ]:
# Angle -> Input -> Quadcell -> mm
# Angle -> Input * Input -> Quadcell * Quadcell -> mm
    # Input -> Angle
pX = [-0.00020920626833899233,  0.005575408543580533, -0.004227915869057122 , 0.015282683828289005 ]
nX = [ 0.005010276070473632  , -0.007508722685443616,  0.0024440587111938927, 0.0011568362685102456]
    # Input -> Quadcell ---> Repeat Several Times to Generate Average?
pM = [[ 0.0283955504013184  ,  0.0                     ,  0.008733329264198737 ,  0.0                  ], 
      [ 0.006408886989959395, -0.0012207403790398877   , -0.0017155674916837088, -0.0003051850947599719], 
      [-0.060731833857234414, -0.0003051850947599719   , -0.02166222113711966  ,  0.0                  ], 
      [ 0.023228095339823618, -0.0013703726310007013   ,  0.0030000000000000027, -0.0009155552842799158]]
nM = [[ 0.17638364818262275 , -0.0015451826532792137   ,  0.047                , -0.001343699453718681 ], 
      [-0.10162663655507063 ,  0.0024414807580797754   , -0.022284432508316295 ,  0.0012207403790398877], 
      [ 0.1235659352397229  ,  0.0                     ,  0.04266222113711966  ,  0.0                  ], 
      [-0.05554368724631491 ,  0.0013703726310007013   , -0.011000000000000003 ,  0.0009155552842799158]]
    # Quadcell -> mm
Qtmm = 3.9

In [ ]:
# Auto-Run Jacobian Input -> QuadCell
pM = [[ 0.0283955504013184  ,  0.0                     ,  0.008733329264198737 ,  0.0                  ], 
      [ 0.006408886989959395, -0.0012207403790398877   , -0.0017155674916837088, -0.0003051850947599719], 
      [-0.060731833857234414, -0.0003051850947599719   , -0.02166222113711966  ,  0.0                  ], 
      [ 0.023228095339823618, -0.0013703726310007013   ,  0.0030000000000000027, -0.0009155552842799158]]

nM = [[ 0.17638364818262275 , -0.0015451826532792137   ,  0.047                , -0.001343699453718681 ], 
      [-0.10162663655507063 ,  0.0024414807580797754   , -0.022284432508316295 ,  0.0012207403790398877], 
      [ 0.1235659352397229  ,  0.0                     ,  0.04266222113711966  ,  0.0                  ], 
      [-0.05554368724631491 ,  0.0013703726310007013   , -0.011000000000000003 ,  0.0009155552842799158]]

In [ ]:
# Auto Jacobian Recorded
PMX = [[ 0.02839555,  0.00640889, -0.06073183,  0.0232281 ],
       [ 0.00873333, -0.00171557, -0.02166222,  0.003     ]]
PMY = [[ 0.        , -0.00122074, -0.00030519, -0.00137037],
       [ 0.        , -0.00030519,  0.        , -0.00091556]]
NMX = [[ 0.17638365, -0.10162664,  0.12356594, -0.05554369],
       [ 0.047     , -0.02228443,  0.04266222, -0.011     ]]
NMY = [[-0.00154518,  0.00244148,  0.        ,  0.00137037],
       [-0.0013437 ,  0.00122074,  0.        ,  0.00091556]]

In [ ]:
# Board Jacobian input -> Angle(rad) Per Mirror
pX = [-0.00020920626833899233,  0.005575408543580533, -0.004227915869057122 , 0.015282683828289005 ]
nX = [ 0.005010276070473632  , -0.007508722685443616,  0.0024440587111938927, 0.0011568362685102456]

In [ ]:
# Detatched Quadcell Jacobian - Input -> Quadcell
pMx = [[ 0.00784741,  0.01098666, -0.0328074,   0.01731553],
       [ 0.00244148,  0.00061037, -0.00656148,  0.0021363]]
nMx = [[-0.072     ,  0.12140294, -0.12963109,  0.03881553],
       [-0.01785333,  0.03125259, -0.03616443,  0.00640518]]
pMy = [[ 0.00030519, -0.00115627, -0.00137333, -0.00106815],
       [ 0.        ,  0.0005    ,  0.        , -0.00010962]]
nYy = [[-0.        ,  0.00015259, -0.00335704, -0.00076296],
       [ 0.00045778,  0.00015184,  0.00034816, -0.00030519]]

In [ ]:

M1 = [0, -2.4,   20.7,  19.4,  41.5,  43,    65,   65.7,   89,     88.54,   11.4] # Step = 14
M2 = [0, 11.5,  -33.5, -25.5, -65.9, -57.7, -97.6, -89.4, -128.8, -120,   -159.7] # Step = 12
M3 = [0, -8.1,   45.5,  28.24, 80.86, 62.2,  112,   93.84, 142.3,  121.1, 168.73] # Step = 30
M4 = [0, 21.42, -7.23,  15.4, -23.14, 8.62, -19.3,  3.24, -26,    -4,        -35] # Step = 12
#     _+S_   _-S_    _+S_   _-S_   _+S_   _-S_   _+S_   _-S_   _+S_    _-S_
#     +[0,1] -[1,2] +[2,3] -[3,4]  +[4,5] -[5,6] +[6,7] -[7,8] +[8,9] -[9,10]

S1 = 14
S2 = 12
S3 = 30
S4 = 12

L2 = 3828 #mm
L4 = 3760 #mm
l1 = np.sqrt(6**2 + 1.605**2)
l3 = np.sqrt((2.5+3.75)**2 +(3.15-1.4)**2)

L1 = L2 + l1
L3 = L4 + l3

L1 = np.array(L1)
L2 = np.array(L2)
L3 = np.array(L3)
L4 = np.array(L4)

# i = constant - finding angle using i(adjacent) & M_(opposite)
# Calc angle wrt "0" - since I is constant, will be no scaling issue
# Figure out how Double Angle theorem ties into this
#   For ever single angular of the mirror, theta change doubles
#   So, divide the movement for 2, to get actual angle change of MIRROR

# Angle change of mirror wrt origin
theta1 = np.atan(M1/(L1))
theta2 = np.atan(M2/(L2))
theta3 = np.atan(M3/(L3))
theta4 = np.atan(M4/(L4))

Theta1 = theta1/(2*S1)
Theta2 = theta2/(2*S2)
Theta3 = theta3/(2*S3)
Theta4 = theta4/(2*S4)

# Individual angle change of mirror
delta1 = np.diff(Theta1)
delta2 = np.diff(Theta2)
delta3 = np.diff(Theta3)
delta4 = np.diff(Theta4)

#     Incremental Angular Changes
# Mirror 1: [-2.23551274e-05  2.15166253e-04 -1.21086977e-05  2.05839940e-04
#   1.39702600e-05  2.04880828e-04  6.51835278e-06  2.16942100e-04
#  -4.28243784e-06 -7.18384915e-04]
# Mirror 2: [ 1.25173779e-04 -4.89802227e-04  8.70724705e-05 -4.39675507e-04
#   8.92313242e-05 -4.34117345e-04  8.92013640e-05 -4.28505704e-04
#   9.56843483e-05 -4.31543129e-04]
# Mirror 3: [-3.58423302e-05  2.37169394e-04 -7.63678045e-05  2.32790144e-04
#  -8.25402983e-05  2.20243316e-04 -8.02976327e-05  2.14221364e-04
#  -9.36948992e-05  2.10447761e-04]
# Mirror 4: [ 0.00023736 -0.00031748  0.00025077 -0.00042708  0.00035195 -0.0003094
#   0.00024978 -0.00032402  0.00024379 -0.00034352]

# Separated Individual Angle Changes of mirror
pos_d1 = delta1[[0, 2, 4, 6, 8]]
neg_d1 = delta1[[1, 3, 5, 7, 9]]

pos_d2 = delta2[[0, 2, 4, 6, 8]]
neg_d2 = delta2[[1, 3, 5, 7, 9]]

pos_d3 = delta3[[0, 2, 4, 6, 8]]
neg_d3 = delta3[[1, 3, 5, 7, 9]]

pos_d4 = delta4[[0, 2, 4, 6, 8]]
neg_d4 = delta4[[1, 3, 5, 7, 9]]

print("Mirror 1")
print(pos_d1)
print(neg_d1)

print("Mirror 2")
print(pos_d2)
print(neg_d2)

print("Mirror 3")
print(pos_d3)
print(neg_d3)

print("Mirror 4")
print(pos_d4)
print(neg_d4)

    # Mirror 1                                                                               - 26.x +
    # [-2.23551274e-05 -1.21086977e-05  1.39702600e-05  6.51835278e-06  -4.28243784e-06]    ~1.18e-05
    # [ 0.00021517  0.00020584  0.00020488  0.00021694 -0.00071838]                         ~3.12e-04  

    # Mirror 2                                                                               - 4.6x +
    # [1.25173779e-04 8.70724705e-05 8.92313242e-05 8.92013640e-05,  9.56843483e-05]        ~9.73e-05
    # [-0.0004898  -0.00043968 -0.00043412 -0.00042851 -0.00043154]                         ~4.45e-04

    # Mirror 3                                                                               - 3.1x +
    # [-3.58423302e-05 -7.63678045e-05 -8.25402983e-05 -8.02976327e-05 -9.36948992e-05]     ~7.38e-05
    # [0.00023717 0.00023279 0.00022024 0.00021422 0.00021045]                              ~2.27e-04

    # Mirror 4                                                                               - 1.3x +
    # [0.00023736 0.00025077 0.00035195 0.00024978 0.00024379]                              ~2.67e-04
    # [-0.00031748 -0.00042708 -0.0003094  -0.00032402 -0.00034352]                         ~3.44e-04

# Average Pos & Neg Change of Mirror
avg_M1p = np.mean(pos_d1)
avg_M1n = np.mean(neg_d1)

avg_M2p = np.mean(pos_d2)
avg_M2n = np.mean(neg_d2)

avg_M3p = np.mean(pos_d3)
avg_M3n = np.mean(neg_d3)

avg_M4p = np.mean(pos_d4)
avg_M4n = np.mean(neg_d4)

plt.figure(figsize=(10, 8))

plt.subplot(2, 2, 1)
plt.title("Mirror 1")
plt.hist(pos_d1, color='red')

plt.subplot(2, 2, 2)
plt.title("Mirror 2")
plt.hist(pos_d2, color='blue')

plt.subplot(2, 2, 3)
plt.title("Mirror 3")
plt.hist(pos_d3, color='green')

plt.subplot(2, 2, 4)
plt.title("Mirror 4")
plt.hist(pos_d4, color='orange')

plt.tight_layout()
plt.show()

plt.hist([pos_d1, pos_d2, pos_d3, pos_d4],
        color=['red', 'blue', 'green', 'orange'],
        label=['Mirror 1', 'Mirror 2', 'Mirror 3', 'Mirror 4'],
        alpha=0.6)
plt.legend()
plt.xlabel("Movement(rad)")
plt.ylabel("Frequency")
plt.title("Positive Inputs")
plt.show()

plt.figure(figsize=(10, 8))

plt.subplot(2, 2, 1)
plt.title("Mirror 1")
plt.hist(neg_d1, color='red')

plt.subplot(2, 2, 2)
plt.title("Mirror 2")
plt.hist(neg_d2, color='blue')

plt.subplot(2, 2, 3)
plt.title("Mirror 3")
plt.hist(neg_d3, color='green')

plt.subplot(2, 2, 4)
plt.title("Mirror 4")
plt.hist(neg_d4, color='orange')

plt.tight_layout()
plt.show()

plt.hist([neg_d1, neg_d2, neg_d3, neg_d4],
        color=['red', 'blue', 'green', 'orange'],
        label=['Mirror 1', 'Mirror 2', 'Mirror 3', 'Mirror 4'])
plt.legend()
plt.xlabel("Movement(rad)")
plt.ylabel("Frequency")
plt.title("Negative Inputs")
plt.show()

# print("Average Change of Mirror 1 - Pos:", avg_M1p, "Neg:", avg_M1n, "in radians")
# print("Average Change of Mirror 2 - Pos:", avg_M2p, "Neg:", avg_M2n, "in radians")
# print("Average Change of Mirror 3 - Pos:", avg_M3p, "Neg:", avg_M3n, "in radians")
# print("Average Change of Mirror 4 - Pos:", avg_M4p, "Neg:", avg_M4n, "in radians")

    # Average Change of Mirror 1 - Pos: -0.00020920626833899233 Neg: 0.005010276070473632 in degrees
    # Average Change of Mirror 2 - Pos: 0.005575408543580533 Neg: -0.007508722685443616 in degrees
    # Average Change of Mirror 3 - Pos: -0.004227915869057122 Neg: 0.0024440587111938927 in degrees
    # Average Change of Mirror 4 - Pos: 0.015282683828289005 Neg: 0.0011568362685102456 in degrees

In [ ]:
A = ops.quads.get_xy_position(sig_strength=0.00)
print(A)
print(A[-2:])
# pQ.append(A[-2:])
nQ.append(A[-2:])

print(pQ)
print(nQ)

pM = []
pM.append([a - b for a, b in zip(pQ[1], pQ[0])])
pM.append([a - b for a, b in zip(pQ[3], pQ[2])])
pM.append([a - b for a, b in zip(pQ[5], pQ[4])])
pM.append([a - b for a, b in zip(pQ[7], pQ[6])])
print(pM)

nM = []
nM.append([a - b for a, b in zip(nQ[1], nQ[0])])
nM.append([a - b for a, b in zip(nQ[3], nQ[2])])
nM.append([a - b for a, b in zip(nQ[5], nQ[4])])
nM.append([a - b for a, b in zip(nQ[7], nQ[6])])
print(nM)

In [ ]:
print("nQ1:", nQ1)
print("pQ1:", pQ1)

print("nQ2:", nQ2)
print("pQ2:", pQ2)

print("nQ3:", nQ3)
print("pQ3:", pQ3)

print("nQ4:", nQ4)
print("pQ4:", pQ4)

pM = np.array(pM*2/2) # Doubling 'cause of mirror law & dividing by initial step jump: pM is the linear movement of laser per singular input
nM = np.array(nM*2/-2)
print("  dx,                dy")
print(pM)
print(nM)

In [ ]:
adjacent = np.array([34.217, 32.907, 34.006, 33.9235])

# Output of Radians
pD0x = np.atan(pM[:,0] / adjacent)
pD0y = np.atan(pM[:,1] / adjacent)
nD0x = np.atan(nM[:,0] / adjacent)
nD0y = np.atan(nM[:,1] / adjacent)

# Change in Mirror Theta per unit input
print("pD0x:", pD0x)
print("pD0y:", pD0y)
print("nD0x:", nD0x)
print("nD0y:", nD0y)

In [ ]:
pD0x = np.array([-3.40955175e-06 , 1.96843993e-03,  2.98910414e-03, 5.62786487e-03])
pD0y = np.array([ 8.91910728e-05, -3.03886711e-05, -7.79601363e-05, -1.50414773e-04])
nD0x = np.array([0.0030287,  0.00731487, 0.00888509, 0.00847054])
nD0y = np.array([-6.91230815e-05, -1.42410204e-04, -2.01358800e-04, -3.82068807e-04])

In [ ]:
# Hopefully: pM = x1, y1, x2, y2

pM = np.array(pM)
nM = np.array(nM)

# 1x4 column vector
pB1x = pM[:,0] # All X values for Beta 1
pB1y = pM[:,1] # All Y values for Beta 1
pB2x = pM[:,2] # All X values for Beta 2
pB2y = pM[:,3] # All Y values for Beta 2

nB1x = nM[:,0] # All X values for Beta 1
nB1y = nM[:,1] # All Y values for Beta 1
nB2x = nM[:,2] # All X values for Beta 2
nB2y = nM[:,3] # All Y values for Beta 2

## Compute Jacobian
pJx = np.zeros((2, 4))
pJy = np.zeros((2, 4))
nJx = np.zeros((2, 4))
nJy = np.zeros((2, 4))

## Positive X Jacobian
pJx[0,:] = pB1x/2
pJx[1,:] = pB2x/2

## Positive Y Jacobian
pJy[0,:] = pB1y/2
pJy[1,:] = pB2y/2

## Negative X Jacobian
nJx[0,:] = nB1x/-2
nJx[1,:] = nB2x/-2

## Negative Y Jacobian
nJy[0,:] = nB1y/-2
nJy[1,:] = nB2y/-2

print("Positive X Jacobian")
print(pJx)

print("Negative X Jacobian")
print(nJx)

print("Positive Y Jacobian")
print(pJy)

print("Negative Y Jacobian")
print(nJy)

In [ ]:
pJx = [[0.00784741, 0.01098666, -0.0328074,   0.01731553],
       [0.00244148, 0.00061037, -0.00656148,  0.0021363 ]]

nJx = [[-0.072,       0.12140294, -0.12963109,  0.03881553],
       [-0.01785333,  0.03125259, -0.03616443,  0.00640518]]

pJy = [[ 0.00030519, -0.00115627, -0.00137333, -0.00106815],
       [ 0,           0.0005 ,     0,          -0.00010962]]

nJy = [ [0,          0.00015259, -0.00335704, -0.00076296],
       [ 0.00045778, 0.00015184,  0.00034816, -0.00030519]]

print("Positive X Jacobian")
print(pJx)

print("Negative X Jacobian")
print(nJx)

print("Positive Y Jacobian")
print(pJy)

print("Negative Y Jacobian")
print(nJy)

%%% WARNING %%%

CAUSED CRASH WHEN RAN

In [ ]:
pSx = pJx / pD0x
pSy = pJy / pD0y
nSx = nJx / nD0x
nSy = nJy / nD0y

print(pSx)
print(pSy)
print(nSx)
print(nSy)